<a href="https://colab.research.google.com/github/clam004/case/blob/main/Load_Training_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
# Mount to my google drive which is where ive stored the data
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/case
!ls

Mounted at /content/drive
/content/drive/MyDrive/case
ACES.ipynb		  modelstates  T0_3B_dialog_summarization.ipynb
data			  __pycache__  Training_Data.xlsx
Load_Training_Data.ipynb  sol.ipynb    utils.py


In [22]:
import pandas as pd

df = pd.read_excel('Training_Data.xlsx')

print(df.columns)

df.head()

Index(['Text', 'Label'], dtype='object')


,Text,Label
0,i just fell this morning in my kitchen,emergency
1,i see no reason for living,emergency
2,i fell in love when i was a young man,neutral
3,coffee is my reason for living,neutral
4,my afternoon meal is 3 hours late,emergency


In [60]:
import random

records = df.to_records(index=False)
result = list(records)

random.shuffle(result)

val_list = result[:int(len(result)*.3)]
train_list = result[int(len(result)*.3):]

val_list

[('i enjoy a late night snack', 'neutral'),
 ('I will die if someone tickled me that way ', 'neutral '),
 ('my legs have been swollen for years', 'neutral'),
 ('i like my toast slightly burned', 'neutral'),
 ("I'm having an allergic attack. Help!", 'emergency'),
 ('the tv shows are so violent these days', 'neutral '),
 ('i am finding it harder to breathe today', 'emergency'),
 ('the red sox got run over by the yankees', 'neutral'),
 ('She had me rolling on the floor laughing', 'neutral'),
 ('My cat attacked a bird outside my house', 'neutral '),
 ('i will kill for some nachos', 'neutral '),
 ('i feel out of breath just walking to the kitchen', 'emergency'),
 ('my afternoon meal is 3 hours late', 'emergency'),
 ('i bumped my head with the sidewalk', 'emergency')]

In [61]:
train_list

[("my nurse hasn't come yet and it's been an hour", 'emergency'),
 ('I like the song Killing me softly ', 'neutral'),
 ('i see no reason for living', 'emergency'),
 ('I fell asleep on the couch ', 'neutral '),
 ('I am having a heart attack', 'emergency'),
 ('My grandchildren will give me a heart attack', 'neutral '),
 ('that music makes me feel like throwing up', 'neutral'),
 ('i fell in love when i was a young man', 'neutral'),
 ('i just got run over by a car', 'emergency'),
 ('im dying for some pizza', 'neutral '),
 ('i ran out of my meds today', 'emergency'),
 ("It's very common to have some allergies during summer", 'neutral'),
 ('i feel out out breath walking up the stairs', 'neutral'),
 ('i just fell this morning in my kitchen', 'emergency'),
 ('on the news i saw that someone was hit by a car', 'neutral'),
 ('I slipped in the bathroom', 'emergency'),
 ('I was supposed to go to the grocery but it slipped my mind. ', 'neutral '),
 ('My neighbor hurt my feelings today ', 'neutral ')

In [62]:
random.choices(train_list, k=4)

[('i just fell this morning in my kitchen', 'emergency'),
 ("It's very common to have some allergies during summer", 'neutral'),
 ('coffee is my reason for living', 'neutral'),
 ('i spent all night throwing up', 'emergency')]

In [63]:
def end_punctuation(utter):
    
    if utter[-1] not in ["?","!","."]:
        utter+="."
        
    return utter

def get_detection_prompt(
    query_text,
    few_shot_list,
    example_symbol = 'Text: ', 
    label_symbol = 'Label: ', 
    sample_separator = '###', 
    join_separator = '\n', 
):
    
    detect_prompt = []
    
    for xy in few_shot_list:
        
        x = end_punctuation(xy[0].strip())
        y = end_punctuation(xy[1].strip())
        
        detect_prompt.append(example_symbol+x)
        detect_prompt.append(label_symbol+y)
        detect_prompt.append(sample_separator)

    query_text = end_punctuation(query_text.strip())
    
    detect_prompt.append(example_symbol+query_text)
    detect_prompt.append(label_symbol)
    
    return join_separator.join(detect_prompt)

In [64]:
concat_prompt = ''

for i in range(len(train_list)):

  few_shot_list = random.choices(train_list, k=2)

  prompt = get_detection_prompt(
      query_text = train_list[i][0],
      few_shot_list = few_shot_list,
  )

  prompt += train_list[i][1]
  concat_prompt += prompt+'.\n###\n'

concat_prompt

"Text: My grandchildren will give me a heart attack.\nLabel: neutral.\n###\nText: Without mexican food I will die.\nLabel: neutral.\n###\nText: my nurse hasn't come yet and it's been an hour.\nLabel: emergency.\n###\nText: My grandchildren will give me a heart attack.\nLabel: neutral.\n###\nText: It's very common to have some allergies during summer.\nLabel: neutral.\n###\nText: I like the song Killing me softly.\nLabel: neutral.\n###\nText: i fell in love when i was a young man.\nLabel: neutral.\n###\nText: that music makes me feel like throwing up.\nLabel: neutral.\n###\nText: i see no reason for living.\nLabel: emergency.\n###\nText: i feel out out breath walking up the stairs.\nLabel: neutral.\n###\nText: I cut myself while cooking and its bleeding.\nLabel: emergency.\n###\nText: I fell asleep on the couch.\nLabel: neutral .\n###\nText: on the news i saw that someone was hit by a car.\nLabel: neutral.\n###\nText: My neighbor hurt my feelings today.\nLabel: neutral.\n###\nText: I am

In [65]:
# load dependencies into our python environment
%%capture
! pip install transformers[sentencepiece]

In [66]:
#sys libs
import os
import sys
import random
import time
import re
import json

#string manupulation libs
import re
import string

#data manupulation libs
import numpy as np

#plotting tools
from matplotlib import pyplot as plt 

#torch libs
import torch
print('torch.__version__', torch.__version__)
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

print('torch.cuda.device_count()', torch.cuda.device_count())
print('torch.cuda.empty_cache()', torch.cuda.empty_cache())

#huggingface transformers
import transformers
print('transformers.__version__', transformers.__version__)
from transformers import set_seed
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# seeds
set_seed(42)
np.random.seed(0)
random.seed(0)
torch.manual_seed(0)

%load_ext autoreload
%autoreload 2
%matplotlib inline

torch.__version__ 1.11.0+cu113
torch.cuda.device_count() 1
torch.cuda.empty_cache() None
transformers.__version__ 4.20.1
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
pretrained_model = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(pretrained_model)

In [67]:
class EmotionDataset(Dataset):

    def __init__(self, tuple_list, tokenizer, chunk_size = 256):

        self.input_ids = []
        self.attn_masks = []
        self.input_token_lens = []

        def get_prompt(tuple_list, i):

            few_shot_list = random.choices(tuple_list, k=4)

            prompt = get_detection_prompt(
                query_text = tuple_list[i][0],
                few_shot_list = few_shot_list,
            )

            prompt += tuple_list[i][1]

            encodings_dict = tokenizer(prompt, return_tensors="pt")

            input_ids = encodings_dict['input_ids']    
            attention_mask = encodings_dict['attention_mask']
            input_token_len = input_ids.shape[1]

            return input_ids, attention_mask, input_token_len

        for i in range(len(tuple_list)):
            unfilled_chunk = True

            input_ids = torch.zeros(1, 0)
            attention_mask = torch.zeros(1, 0)
            input_token_len = 0

            while unfilled_chunk:

              input_ids_, attention_mask_, input_token_len_ = \
                get_prompt(tuple_list, i)
              self.input_token_lens.append(input_token_len_)

              input_token_len += input_token_len_
              input_ids = torch.cat([input_ids, input_ids_], dim=1)
              attention_mask = torch.cat([attention_mask, attention_mask_], dim=1)

              if input_token_len > chunk_size:
                input_ids = input_ids[:,:chunk_size]
                attention_mask = attention_mask[:,:chunk_size]
                unfilled_chunk = False

            self.input_ids.append(input_ids.long())
            self.attn_masks.append(attention_mask.long())

        print('mean', np.mean(self.input_token_lens), 'min', min(self.input_token_lens), 'max', max(self.input_token_lens))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [68]:
traindata = EmotionDataset(train_list, tokenizer, chunk_size = 128)

mean 90.48571428571428 min 80 max 103


In [69]:
tokenizer.batch_decode(traindata.__getitem__(0)[0])

["Text: My house is being attacked.\nLabel: emergency.\n###\nText: i fell down on the driveway.\nLabel: emergency.\n###\nText: on the news i saw that someone was hit by a car.\nLabel: neutral.\n###\nText: im dying for some pizza.\nLabel: neutral.\n###\nText: my nurse hasn't come yet and it's been an hour.\nLabel: emergencyText: My neighbor hurt my feelings today.\nLabel: neutral.\n###\nText: on the news i saw that someone was hit by a car.\nLabel: neutral."]

In [71]:
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler

batch_size = 4

# Take training samples in random order. 
train_dataloader = DataLoader(
    traindata,  # The training samples.
    sampler = RandomSampler(traindata), # Select batches randomly. 
    batch_size = batch_size # Trains with this batch size.
)

for batch_idx, samples in enumerate(train_dataloader):

      break

samples[0].squeeze(1).shape

torch.Size([4, 128])

In [37]:
#sys libs
import os
import sys
import random
import time
import re
import json


pretrained_model = 'gpt2'


cache_dir = os.path.join(
    "./modelstates/hugface_models/",
    pretrained_model,
)

print("cache_dir=", cache_dir)

model_save_path = os.path.join(
    "./modelstates/finetuned_models",
    pretrained_model,
)

print("model_save_path=", model_save_path)

model = GPT2LMHeadModel.from_pretrained(
    pretrained_model,
    cache_dir=cache_dir,
)

cache_dir= ./modelstates/hugface_models/gpt2
model_save_path= ./modelstates/finetuned_models/gpt2


In [73]:
class BaseAgent(torch.nn.Module):

    def __init__(self, model, tokenizer):
        
        super().__init__()
        
        self.model = model
        self.tokenizer = tokenizer
        
        self.optimizer = torch.optim.Adam(
            self.model.parameters(),
            lr=0.00001,
            betas=(0.9, 0.98),
            eps=1e-9,
        )
        
        self.num_gpus = torch.cuda.device_count()
        
        if self.num_gpus > 1:
            self.model.parallelize()
        elif self.num_gpus == 1:
            self.gpu0 = torch.device('cuda:0')
            #self.model = self.model.cuda()
            self.model = self.model.to(self.gpu0)
            '''you can do .to(cuda0) with tensors to'''
            
        print('self.model.device', self.model.device)
        
        self.num_params = \
          sum(p.numel() for p in self.model.parameters() if p.requires_grad)
        print("num_params", self.num_params)
        
    def get_response(self, prompt, max_len = 32, no_repeat_ngram_size = None):
        
        prompt_dic = self.tokenizer(prompt,return_tensors="pt")
        prompt_ids = prompt_dic.input_ids
        prompt_mask = prompt_dic.attention_mask
        prompt_len = prompt_ids.shape[1]
        
        if self.num_gpus > 0:
            prompt_ids = prompt_ids.to(self.model.device)
            prompt_mask = prompt_mask.to(self.model.device)
        
        prompt_len = prompt_ids.shape[1]
        
        output_ids = self.model.generate(
            prompt_ids,
            attention_mask = prompt_mask,
            max_length = prompt_len+max_len,
            no_repeat_ngram_size = no_repeat_ngram_size,
        )

        generated_text = self.tokenizer.batch_decode(output_ids)[0]
        
        return generated_text
    
    def memorize(self, prompt, num_epochs = 3):

        print('start training loop')

        """ This is a rudimentary training loop
        that will train the agent to learn one
        sequence, the prompt. With enough epochs, this should
        result in memorizing the sequence, which is why this
        class method was names memorize. 
        There is nothing returned because the model attribute is modified inplace. 
        Args:
            prompt (string): the text to be learned
            num_epochs (int): the number of times we cycle though the training data, only 1 sample in this case
        """

        prompt_dic = self.tokenizer(prompt,return_tensors="pt")
        prompt_ids = prompt_dic.input_ids
        prompt_mask = prompt_dic.attention_mask
        prompt_len = prompt_ids.shape[1]

        if self.num_gpus > 0:
            prompt_ids = prompt_ids.to(self.model.device)
            prompt_mask = prompt_mask.to(self.model.device)
            
        source_ids = prompt_ids[:,:-1]
        target_ids = prompt_ids[:,1:]
        source_mask = prompt_mask[:,:-1]
        target_mask = prompt_mask[:,1:]

        # allow params to be updated
        self.model.train()

        for e in range(num_epochs):

            outputs = self.model(
                input_ids = prompt_ids,
                labels = prompt_ids,
                attention_mask = prompt_mask,
                token_type_ids=None,
            )

            # used logits and target tokens to calculate the loss
            loss = outputs.loss
            logits = outputs.logits

            # backward pass
            self.optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
            self.optimizer.step()

            print("epoch", e, "loss", loss.item())

        torch.cuda.empty_cache()

    
    def train(self, train_dataloader, num_epochs = 3):

      for epoch in range(num_epochs):

        for batch_idx, samples in enumerate(train_dataloader):

          samples = [s.to(self.model.device) for s in samples]

          outputs = self.model(
              input_ids = samples[0].squeeze(1),
              labels = samples[0].squeeze(1),
              attention_mask = samples[1].squeeze(1),
              token_type_ids=None,
          )

          # used logits and target tokens to calculate the loss
          loss = outputs.loss
          logits = outputs.logits

          # backward pass
          self.optimizer.zero_grad()
          loss.backward()
          torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
          self.optimizer.step()

        print("epoch", epoch, "loss", loss.item())

      torch.cuda.empty_cache()

    def eval(self, val_list):

      for sample in val_list:

        print(sample)



In [74]:
agent = BaseAgent(model, tokenizer)

self.model.device cuda:0
num_params 124439808


In [75]:
agent.eval(val_list)

('i enjoy a late night snack', 'neutral')
('I will die if someone tickled me that way ', 'neutral ')
('my legs have been swollen for years', 'neutral')
('i like my toast slightly burned', 'neutral')
("I'm having an allergic attack. Help!", 'emergency')
('the tv shows are so violent these days', 'neutral ')
('i am finding it harder to breathe today', 'emergency')
('the red sox got run over by the yankees', 'neutral')
('She had me rolling on the floor laughing', 'neutral')
('My cat attacked a bird outside my house', 'neutral ')
('i will kill for some nachos', 'neutral ')
('i feel out of breath just walking to the kitchen', 'emergency')
('my afternoon meal is 3 hours late', 'emergency')
('i bumped my head with the sidewalk', 'emergency')


In [59]:
agent.train(train_dataloader)

epoch 0 loss 1.1636629104614258
epoch 1 loss 0.6864692568778992
epoch 2 loss 0.36334311962127686
